In [202]:
pip install pymysql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [203]:
from sqlalchemy import create_engine
import pymysql
import pandas as pd

In [204]:
# db 연결
endpoint = 'dao.c51deksujiip.ap-northeast-2.rds.amazonaws.com'
schema_name = 'dao'
db_connection_str = 'mysql+pymysql://admin:ekfkawnl@{}/{}'.format(endpoint, schema_name )
try :
    db_connection = create_engine(db_connection_str)
    conn = db_connection.connect()
except :
    print('fail to connect db')

In [205]:
# df_main 생성 : maindata
sql = "SELECT * FROM {}".format('MainData_현수')
df_main=pd.read_sql(sql,db_connection)
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 509 entries, 0 to 508
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   번호                       509 non-null    int64  
 1   생태통로명                    385 non-null    object 
 2   주소                       509 non-null    object 
 3   도로명                      500 non-null    object 
 4   도로번호                     488 non-null    object 
 5   차선_수                     509 non-null    object 
 6   위도_도분초                   509 non-null    object 
 7   경도_도분초                   509 non-null    object 
 8   위도_십진법                   509 non-null    float64
 9   경도_십진법                   509 non-null    float64
 10  num                      497 non-null    float64
 11  도로                       496 non-null    object 
 12  생태통로타입                   497 non-null    object 
 13  준공년도                     494 non-null    object 
 14  모니터링기간                   4

In [206]:
# df1 생성 : 국립생태원 규격 data
df1=pd.read_csv('/content/drive/MyDrive/생태통로_국립생태원PDF.csv')
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 0 to 65
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   위도      66 non-null     float64
 1   경도      66 non-null     float64
 2   연장(m)   66 non-null     float64
 3   폭(m)    65 non-null     float64
 4   높이(m)   20 non-null     float64
 5   개방도     20 non-null     float64
 6   이용확인종   61 non-null     object 
dtypes: float64(6), object(1)
memory usage: 3.7+ KB


In [207]:
# df2 생성 : 환경부 규격 data
df2=pd.read_csv('/content/drive/MyDrive/생태통로_환경부PDF.csv')
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   위도      152 non-null    float64
 1   경도      152 non-null    float64
 2   연장(m)   152 non-null    float64
 3   폭(m)    152 non-null    float64
 4   높이(m)   30 non-null     float64
 5   개방도     30 non-null     float64
dtypes: float64(6)
memory usage: 7.2 KB


In [208]:
# df_main 열 이름 df1, df2와 동일하게 변경
df_main.rename(columns = {'위도_십진법':'위도',
                          '경도_십진법':'경도',
                          '너비_m':'폭(m)',
                          '길이_m':'연장(m)'},inplace=True)

# df_main 불필요한 '면적_m^2' 열 제거
df_main.drop('면적_m^2', axis=1, inplace=True)

In [209]:
# df_main과 위도, 경도 비교해 df에 일치하는 '번호' 열 추가
def append_num(df_main, df, col, col_list):
  df=df.reindex(columns = df.columns.tolist() + [col])
  for i in range(len(df)):
    x=df.loc[i, col_list[0]]
    y=df.loc[i, col_list[1]]
    for j in range(len(df_main)):
      x_main=df_main.loc[j, col_list[0]]
      y_main=df_main.loc[j, col_list[1]]
      if x==x_main and y==y_main:
        df.loc[i, col]=df_main.loc[j, col]
  df.drop(col_list, axis=1, inplace=True)
  return df

# df1에 '번호' 열 추가
df1 = append_num(df_main,df1,'번호',['위도','경도'])
df1 = df1.reindex(['번호', '연장(m)', '폭(m)', '높이(m)', '개방도'], axis=1)
#df1.info()

# df2에 '번호' 열 추가
df2 = append_num(df_main,df2,'번호',['위도','경도'])
df2 = df2.reindex(['번호', '연장(m)', '폭(m)', '높이(m)', '개방도'], axis=1)
#df2.info()

In [210]:
# df_main과 df를 '번호' 열을 기준으로 병합하는 함수
def merge_df(df_main, df, col_list):
  return pd.merge(df_main[[col_list[0],col_list[1],col_list[2],col_list[3],col_list[4]]], df[[col_list[0],col_list[1],col_list[2],col_list[3],col_list[4]]], how='left', on=col_list[0])

# df_main의 생태통로 규격 data(폭, 연장, 높이, 개방도)가 비어있으면, df의 규격 data를 입력하는 함수
def change_size(df_main, df, col_list):
  coord =merge_df(df_main, df, col_list)
  idx = list(df_main[((df_main[col_list[1]].isnull()) | df_main[col_list[2]].isnull())].index)
  for i in idx:
    for j in range(1, len(col_list)):
      df_main.loc[i, col_list[j]]=coord.loc[i, col_list[j]+'_y']

# df_main에 '높이(m)', '개방도' 열 추가
df_main=df_main.reindex(columns = df_main.columns.tolist() + ['높이(m)','개방도'])

# df1로 df_main 규격 data 수정
change_size(df_main, df1, ['번호','폭(m)','연장(m)', '높이(m)', '개방도'])
df_main.info()

# df2로 df_main 규격 data 수정
change_size(df_main, df2, ['번호','폭(m)','연장(m)', '높이(m)', '개방도'])
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 509 entries, 0 to 508
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   번호                       509 non-null    int64  
 1   생태통로명                    385 non-null    object 
 2   주소                       509 non-null    object 
 3   도로명                      500 non-null    object 
 4   도로번호                     488 non-null    object 
 5   차선_수                     509 non-null    object 
 6   위도_도분초                   509 non-null    object 
 7   경도_도분초                   509 non-null    object 
 8   위도                       509 non-null    float64
 9   경도                       509 non-null    float64
 10  num                      497 non-null    float64
 11  도로                       496 non-null    object 
 12  생태통로타입                   497 non-null    object 
 13  준공년도                     494 non-null    object 
 14  모니터링기간                   4

In [211]:
# df → db
# table_name1 = 'MainData_하영'
# table_name2 = '생태통로_규격_국립생태원PDF'
# table_name3 = '생태통로_규격_환경부PDF'
# #df_main.to_sql(name=table_name1, con=db_connection, if_exists='fail', index=False)
# df1.to_sql(name=table_name2, con=db_connection, if_exists='fail', index=False)
# df2.to_sql(name=table_name3, con=db_connection, if_exists='fail', index=False)